<a href="https://colab.research.google.com/github/YOUR_USERNAME/calvano-thesis/blob/v1.1-parallel/calvano_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎯 Calvano et al. (2020) Full Experiment

This notebook runs the full-scale experiment (50,000 episodes × 10 seeds) for replicating Calvano et al. (2020) Table A.2.

**Estimated Runtime:**
- Colab Free (2 cores): ~3 hours
- Colab Pro (4 cores): ~1.5 hours

**Features:**
- ✅ Parallel execution with 4 workers
- ✅ Resume capability (survives disconnections)
- ✅ Google Drive backup
- ✅ Automatic result download

In [ ]:
# 📦 Setup: Clone repository and install dependencies
!git clone --branch v1.1-parallel https://github.com/YOUR_USERNAME/calvano-thesis.git
%cd calvano-thesis
!pip install -q -r requirements.txt

print("✅ Setup complete!")
!python -c "import src.env; print('Environment test:', src.env.DemandEnvironment().get_nash_equilibrium()['individual_profit'])"

In [ ]:
# 💾 Optional: Mount Google Drive for backup
from google.colab import drive
import os

drive.mount('/content/drive')
OUT_DIR = "/content/drive/MyDrive/calvano_results"
os.makedirs(OUT_DIR, exist_ok=True)

print(f"📁 Results will be saved to: {OUT_DIR}")

In [ ]:
# 🚀 Run Full Experiment: 50,000 episodes × 10 seeds
import time

print("🎯 Starting full experiment...")
print("📊 Target: Individual ~0.18±0.03, Joint ~0.26±0.04")
print("⏰ Estimated time: 1.5-3 hours")
print()

start_time = time.time()

!python scripts/table_a2_parallel.py \
    --episodes 50000 \
    --n-seeds 10 \
    --workers 4 \
    --out-dir $OUT_DIR \
    --csv $OUT_DIR/table_a2_full.csv \
    --json $OUT_DIR/table_a2_full.json \
    --resume

elapsed = (time.time() - start_time) / 3600
print(f"\n🏁 Experiment completed in {elapsed:.2f} hours")

In [ ]:
# 🔄 Resume Experiment (run this cell if disconnected)
print("🔄 Resuming experiment...")

!python scripts/table_a2_parallel.py \
    --episodes 50000 \
    --n-seeds 10 \
    --workers 4 \
    --out-dir $OUT_DIR \
    --csv $OUT_DIR/table_a2_full.csv \
    --json $OUT_DIR/table_a2_full.json \
    --resume

In [ ]:
# �� Analyze Results
import pandas as pd
import json
import glob

# Load individual seed results
seed_files = glob.glob(f"{OUT_DIR}/seed_*.csv")
if seed_files:
    df = pd.concat([pd.read_csv(f) for f in seed_files])
    print(f"📈 Results from {len(df)} seeds:")
    print(df[['seed', 'individual_profit', 'joint_profit']].round(4))
    print()
    
    # Summary statistics
    individual_mean = df['individual_profit'].mean()
    individual_std = df['individual_profit'].std()
    joint_mean = df['joint_profit'].mean()
    joint_std = df['joint_profit'].std()
    
    print(f"📊 Summary Statistics:")
    print(f"   Individual: {individual_mean:.4f} ± {individual_std:.4f}")
    print(f"   Joint: {joint_mean:.4f} ± {joint_std:.4f}")
    print()
    
    # Compare to targets
    target_individual = 0.18
    target_joint = 0.26
    individual_diff = abs(individual_mean - target_individual)
    joint_diff = abs(joint_mean - target_joint)
    
    print(f"🎯 Comparison to Calvano et al. (2020):")
    print(f"   Individual: {'✅' if individual_diff <= 0.03 else '⚠️'} {individual_diff:.4f} from target")
    print(f"   Joint: {'✅' if joint_diff <= 0.04 else '⚠️'} {joint_diff:.4f} from target")
    
    # Save combined results
    df.to_csv("results/table_a2_full_combined.csv", index=False)
    print(f"\n💾 Combined results saved to results/table_a2_full_combined.csv")
else:
    print("❌ No results found. Please run the experiment first.")

In [ ]:
# 📥 Download Results to Local Machine
from google.colab import files
import os

# Download main result files
result_files = [
    "results/table_a2_full_combined.csv",
    f"{OUT_DIR}/table_a2_full.json",
    f"{OUT_DIR}/table_a2_full.csv"
]

for file_path in result_files:
    if os.path.exists(file_path):
        print(f"📥 Downloading {file_path}...")
        files.download(file_path)
    else:
        print(f"⚠️ File not found: {file_path}")

print("\n✅ Download complete!")
print("📁 Files are also backed up in your Google Drive at:", OUT_DIR)

## 🔧 Troubleshooting

**If the experiment gets interrupted:**
1. Run the "Resume Experiment" cell
2. The `--resume` flag will skip completed seeds
3. Only remaining seeds will be processed

**If you need to restart:**
1. Re-run the setup cell
2. Re-mount Google Drive
3. Run the resume experiment cell

**For different experiment sizes:**
- Quick test: `--episodes 1000 --n-seeds 2`
- Medium scale: `--episodes 1000 --n-seeds 5`
- Full scale: `--episodes 50000 --n-seeds 10`